In [3]:
import pandas as pd

# Read in the data
unzipped_path = os.path.abspath(os.path.join(".","..","data","m5-forecasting-accuracy-unzipped"))
INPUT_DIR = unzipped_path
WORKING_DIR = os.path.join(unzipped_path, "..", "working")

hierarchichal = pd.read_parquet(f"{WORKING_DIR}/hierarchical.parquet")

In [4]:
hierarchichal

,unique_id,item_id,dept_id,cat_id,store_id,state_id,ds,y
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-28,0
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,0
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-30,0
3,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-31,0
4,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-02-01,0
...,...,...,...,...,...,...,...,...
59181085,HOUSEHOLD_1_205_TX_3_evaluation,HOUSEHOLD_1_205,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,2016-05-17,3
59181086,HOUSEHOLD_1_205_TX_3_evaluation,HOUSEHOLD_1_205,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,2016-05-18,2
59181087,HOUSEHOLD_1_205_TX_3_evaluation,HOUSEHOLD_1_205,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,2016-05-19,5
59181088,HOUSEHOLD_1_205_TX_3_evaluation,HOUSEHOLD_1_205,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,2016-05-20,4


In [6]:
import numpy as np
import pandas as pd

#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData

# Load TourismSmall dataset
Y_df, S, tags = HierarchicalData.load('./data', 'TourismSmall')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

ModuleNotFoundError: No module named 'numpy'

In [ ]:

#split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(12)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_test_df  = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

# Compute base auto-ARIMA predictions
fcst = StatsForecast(df=Y_train_df, 
                     models=[AutoARIMA(season_length=12), Naive()], 
                     freq='M', n_jobs=-1)
Y_hat_df = fcst.forecast(h=12)

# Reconcile the base predictions
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/Purpose/State',
              top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, 
                          S=S, tags=tags)

In [ ]:
# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut